## 라이브러리

In [5]:
import os
import json
from collections import defaultdict,Counter

## [Korquad](https://korquad.github.io/KorQuad%201.0/) evaluate.py의 후처리 함수 -> 사용하려면 밑의 @@@ 부분 주석 해제

In [6]:
import re
import string
def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [7]:
file_path ='./nbest'                                  #소프트보팅 할 nbest_predictions.json들 있는 경로
n=20                                                  #nbest prediction -> 20개로저장(utils_qa.py의 default nbest_size)
json_file_path = []
json_files=[]
for json_path in os.listdir(file_path):
    if json_path.endswith('.json'):                    #checkpoint 및 하위 폴더들 ignore (KLUE대회 태일님 앙상블 참고)
        json_file_path.append(json_path)

for file in json_file_path:
    fp=os.path.join(file_path,file)
    with open(fp,"r",encoding='utf-8') as json_file:
        json_data=json.load(json_file)
        json_files.append(json_data)

In [8]:
key_list=list(json_files[0].keys())                   # predictions.json의 key값들 설정
bin=defaultdict(list)                                 # 확률값들을 담을 dict생성-> value : list
answer_dict=defaultdict(list)                         # 최종 답변을 넣을 dict 생성 -> value : list

for i in range(len(key_list)):                        #defaultdict : key만 선언하면 value는 '' 빈 값
    bin[key_list[i]]                                 
    answer_dict[key_list[i]]

for json_file in json_files:                          # N개의 nbest_predictions 파일들
    for kl in key_list:                               # 600개의 key
        for i in range(n):                            # nbest_size : 20                       
            bin[kl].append((json_file[kl][i]['text'],json_file[kl][i]['probability']))                        
            # bin[kl].append((normalize_answer(json_file[kl][i]['text']),json_file[kl][i]['probability']))    #@@@@@@후처리 함수 적용@@@@@
            
                #@@@@@@@@후처리 함수 적용 후 소프트보팅 하려면 위 코드 주석해제 후 바로위 코드 주석처리 (위의 korquad 후처리 함수) @@@@@@

In [9]:
for kl in key_list:                                   # 600개의 키
    answer_hubo={}
    for i in range(len(bin[kl])):                  
        if bin[kl][i][0] not in answer_hubo.keys():   # 빈 객체에서 없다면 key 추가
            answer_hubo[bin[kl][i][0]]=bin[kl][i][1]
        else:
            answer_hubo[bin[kl][i][0]]+=bin[kl][i][1] # 있다면 확률값 더하기 -> 이렇게 하지 않으면 dictionary는 자동으로 중복제거(set)
    max_answer=max(answer_hubo.keys(),key= lambda prob : answer_hubo[prob])     #최대 확률값을 답변으로
    answer_dict[kl]=max_answer

In [10]:
new_nbest_json_path = "nbest_ensemble.json"           # nbest_ensemble.json 저장

with open(new_nbest_json_path, 'w', encoding='utf-8') as file:
    json.dump(answer_dict, file)